use this notebook to see today's bet at the beginning of a day

In [103]:
import pandas as pd, numpy as np
import statsapi
from statsapi import player_stat_data
import requests
from datetime import datetime, timedelta
import numpy as np
import math
import meteostat
import pprint
import pickle
import copy

import pycaret
from pycaret import classification
import model.common
import importlib

In [19]:
from static_data.load_static_data import *

In [3]:
date_today = datetime.today().strftime("%Y-%m-%d")
date_yesterday = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
date_today, date_yesterday

('2023-05-29', '2023-05-28')

In [58]:
importlib.reload(model.common)

<module 'model.common' from '/home/junlim/projects/mlb-props/model/common.py'>

In [86]:
df_live_game_matchup = pd.read_pickle(f'collect_data/df_live_game_matchup_{date_today}.pkl')
df_odds_hits = pd.read_pickle(f"odds_data/df_odds_hits_{date_today}.pkl")

In [87]:
df_odds_hits["over_prob"] = df_odds_hits["over_odds"].apply(model.common.odds_to_probability)

In [88]:
df_live_game_matchup[model.common.features + ['game_id']].set_index(['game_id', 'batting_name']).join(df_odds_hits.rename(columns={'player_name': 'batting_name'}).set_index(['game_id', 'batting_name']), lsuffix='', rsuffix='_odds')

pitching_gamesPlayed  pitching_runs  \
game_id batting_name                                             
717974  Adam Frazier                          4              3   
        Gunnar Henderson                      4              3   
        Jorge Mateo                           4              3   
        Anthony Santander                     4              3   
        Ryan McKenna                          4              3   
...                                         ...            ...   
717971  Hunter Renfroe                       25             53   
        Brandon Drury                        25             53   
        Mike Trout                           25             53   
        Gio Urshela                          25             53   
        Shohei Ohtani                        25             53   

                           pitching_strikeOuts  pitching_hits  pitching_id  \
game_id batting_name                                                         
717974  Adam Frazier                         6              9       663531   
        Gunnar Henderson                     6              9       663531   
        Jorge Mateo                          6              9       663531   
        Anthony Santander                    6              9       663531   
        Ryan McKenna                         6              9       663531   
...                                        ...            ...          ...   
717971  Hunter Renfroe                     105             85       656629   
        Brandon Drury                      105             85       656629   
        Mike Trout                         105             85       656629   
        Gio Urshela                        105             85       656629   
        Shohei Ohtani                      105             85       656629   

                           batting_gamesPlayed  batting_runs  \
game_id batting_name                                           
717974  Adam Frazier                       156            61   
        Gunnar Henderson                    34            12   
        Jorge Mateo                        150            63   
        Anthony Santander                  152            78   
        Ryan McKenna                       105            23   
...                                        ...           ...   
717971  Hunter Renfroe                     125            62   
        Brandon Drury                      138            87   
        Mike Trout                         119            85   
        Gio Urshela                        144            61   
        Shohei Ohtani                      157            90   

                           batting_strikeOuts  batting_hits  batting_rbi  ...  \
game_id batting_name                                                      ...   
717974  Adam Frazier                       73           129           42  ...   
        Gunnar Henderson                   34            30           18  ...   
        Jorge Mateo                       147           109           50  ...   
        Anthony Santander                 122           138           89  ...   
        Ryan McKenna                       55            37           11  ...   
...                                       ...           ...          ...  ...   
717971  Hunter Renfroe                    121           121           72  ...   
        Brandon Drury                     126           136           87  ...   
        Mike Trout                        139           124           80  ...   
        Gio Urshela                        96           143           64  ...   
        Shohei Ohtani                     161           160           95  ...   

                           game_year   pitching_name  game_date_odds  \
game_id batting_name                                                   
717974  Adam Frazier            2023     Logan Allen      2023-05-29   
        Gunnar Henderson        2023     Logan All

# live prediction

In [27]:
regression_model = pycaret.classification.load_model(model.common.model_file_name)

Transformation Pipeline and Model Successfully Loaded


In [89]:
live_prediction = pycaret.classification.predict_model(data = df_live_game_matchup, estimator = regression_model)
live_prediction = pd.merge(live_prediction, df_player_team_positions[['player_id','player_team_name']], left_on='batting_id', right_on='player_id', how='left')
live_prediction["theo_odds"] = live_prediction["prediction_score"].apply(model.common.odds_calculator)

In [90]:
live_bet_columns = model.common.features + ['prediction_label', 'prediction_score', 'theo_odds']
live_prediction[live_bet_columns]

,pitching_gamesPlayed,pitching_runs,pitching_strikeOuts,pitching_hits,pitching_id,batting_gamesPlayed,batting_runs,batting_strikeOuts,batting_hits,batting_rbi,...,batting_id,temp,game_venue,game_date,game_year,batting_name,pitching_name,prediction_label,prediction_score,theo_odds
0,4,3,6,9,663531,156,61,73,129,42,...,624428,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Adam Frazier,Logan Allen,1,0.6346,-174
1,4,3,6,9,663531,156,61,73,129,42,...,624428,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Adam Frazier,Logan Allen,1,0.6346,-174
2,4,3,6,9,663531,156,61,73,129,42,...,624428,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Adam Frazier,Logan Allen,1,0.6346,-174
3,4,3,6,9,663531,156,61,73,129,42,...,624428,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Adam Frazier,Logan Allen,1,0.6346,-174
4,4,3,6,9,663531,34,12,34,30,18,...,683002,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Gunnar Henderson,Logan Allen,1,0.5512,-123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,25,53,105,85,656629,144,61,96,143,64,...,570482,77.180000,Guaranteed Rate Field,2023-05-29,2023,Gio Urshela,Michael Kopech,1,0.5492,-122
196,25,53,105,85,656629,144,61,96,143,64,...,570482,77.180000,Guaranteed Rate Field,2023-05-29,2023,Gio Urshela,Michael Kopech,1,0.5492,-122
197,25,53,105,85,656629,144,61,96,143,64,...,570482,77.180000,Guaranteed Rate Field,2023-05-29,2023,Gio Urshela,Michael Kopech,1,0.5492,-122
198,25,53,105,85,656629,144,61,96,143,64,...,570482,77.180000,Guaranteed Rate Field,2023-05-29,2023,Gio Urshela,Michael Kopech,1,0.5492,-122


In [99]:
df_live_prediction_odds = live_prediction[live_bet_columns + ['game_id']].set_index(['game_id', 'batting_name']).join(df_odds_hits.rename(columns={'player_name': 'batting_name'}).set_index(['game_id', 'batting_name']), lsuffix='', rsuffix='_odds').reset_index()
df_live_prediction_odds = df_live_prediction_odds[df_live_prediction_odds.over_line < 1.0]
df_live_prediction_odds

,game_id,batting_name,pitching_gamesPlayed,pitching_runs,pitching_strikeOuts,pitching_hits,pitching_id,batting_gamesPlayed,batting_runs,batting_strikeOuts,...,prediction_score,theo_odds,game_date_odds,team_away,team_home,over_odds,over_line,under_odds,under_line,over_prob
0,717968,Alex Bregman,24,44,117,99,543243,155,93,77,...,0.6747,-207,2023-05-29,Minnesota Twins,Houston Astros,-225,0.5,+165,0.5,0.692308
1,717968,Jake Meyers,24,44,117,99,543243,52,13,54,...,0.5284,-112,2023-05-29,Minnesota Twins,Houston Astros,-165,0.5,+125,0.5,0.622642
2,717968,Jeremy Pena,24,44,117,99,543243,136,72,135,...,0.5844,-141,2023-05-29,Minnesota Twins,Houston Astros,-225,0.5,+165,0.5,0.692308
3,717968,Jose Abreu,24,44,117,99,543243,157,85,110,...,0.7370,-280,2023-05-29,Minnesota Twins,Houston Astros,-260,0.5,+180,0.5,0.722222
4,717968,Jose Abreu,24,44,117,99,543243,157,85,110,...,0.7370,-280,2023-05-29,Minnesota Twins,Houston Astros,-260,0.5,+180,0.5,0.722222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,717979,Robbie Grossman,10,2,13,5,571510,129,40,129,...,0.5817,-139,2023-05-29,Texas Rangers,Detroit Tigers,-150,0.5,+115,0.5,0.600000
189,717979,Spencer Torkelson,20,55,103,115,543135,110,38,99,...,0.6021,-151,2023-05-29,Texas Rangers,Detroit Tigers,-220,0.5,+160,0.5,0.687500
196,717979,Zach McKinstry,20,55,103,115,543135,57,21,52,...,0.6168,-161,2023-05-29,Texas Rangers,Detroit Tigers,-240,0.5,+175,0.5,0.705882
197,717979,Zach McKinstry,20,55,103,115,543135,57,21,52,...,0.6168,-161,2023-05-29,Texas Rangers,Detroit Tigers,-240,0.5,+175,0.5,0.705882


In [106]:
score_threshold = 0.70
df_live_confident_prediction_odds = df_live_prediction_odds[(df_live_prediction_odds["prediction_score"] >= score_threshold)].sort_values(by = "prediction_score", ascending = False).drop_duplicates("batting_name")
df_live_confident_prediction_odds[['game_date', 'team_away', 'team_home', 'batting_name', "prediction_score", "theo_odds", 'over_prob', 'over_odds', 'over_line']]

,game_date,team_away,team_home,batting_name,prediction_score,theo_odds,over_prob,over_odds,over_line
61,2023-05-29,Cleveland Guardians,Baltimore Orioles,Ryan McKenna,0.7817,-358,0.649123,-185,0.5
5,2023-05-29,Minnesota Twins,Houston Astros,Jose Altuve,0.7475,-296,0.722222,-260,0.5
157,2023-05-29,Texas Rangers,Detroit Tigers,Adolis Garcia,0.7387,-283,0.696970,-230,0.5
3,2023-05-29,Minnesota Twins,Houston Astros,Jose Abreu,0.7370,-280,0.722222,-260,0.5
113,2023-05-29,Colorado Rockies,Arizona Diamondbacks,Kris Bryant,0.7365,-280,0.718310,-255,0.5
176,2023-05-29,Texas Rangers,Detroit Tigers,Marcus Semien,0.7352,-278,0.729730,-270,0.5
161,2023-05-29,Texas Rangers,Detroit Tigers,Corey Seager,0.7281,-268,0.722222,-260,0.5
36,2023-05-29,Los Angeles Angels,Chicago White Sox,Mickey Moniak,0.7262,-265,0.574468,-135,0.5
165,2023-05-29,Texas Rangers,Detroit Tigers,Javier Baez,0.7135,-249,0.733333,-275,0.5
82,2023-05-29,Kansas City Royals,St. Louis Cardinals,Paul Goldschmidt,0.7073,-242,0.710145,-245,0.5


In [111]:
advantageous_score_threshold = 0.60
prediction_diff_threshold = 0.05
df_live_prediction_odds_ = copy.copy(df_live_prediction_odds)
df_live_prediction_odds_['prediction_diff'] = df_live_prediction_odds_['prediction_score'] - df_live_prediction_odds_['over_prob']
df_live_advantageous_prediction_odds = df_live_prediction_odds_.sort_values(by = "prediction_diff", ascending = False).drop_duplicates("batting_name")
df_live_advantageous_prediction_odds = df_live_advantageous_prediction_odds[(df_live_advantageous_prediction_odds["prediction_score"] >= advantageous_score_threshold)]
df_live_advantageous_prediction_odds = df_live_advantageous_prediction_odds[(df_live_advantageous_prediction_odds["prediction_diff"] >= prediction_diff_threshold)]
df_live_advantageous_prediction_odds[['game_date', 'team_away', 'team_home', 'batting_name', "prediction_diff", "prediction_score", "theo_odds", 'over_prob', 'over_odds', 'over_line']]

,game_date,team_away,team_home,batting_name,prediction_diff,prediction_score,theo_odds,over_prob,over_odds,over_line
34,2023-05-29,Los Angeles Angels,Chicago White Sox,Matt Thaiss,0.165190,0.6890,-222,0.523810,-110,0.5
35,2023-05-29,Los Angeles Angels,Chicago White Sox,Mickey Moniak,0.151732,0.7262,-265,0.574468,-135,0.5
61,2023-05-29,Cleveland Guardians,Baltimore Orioles,Ryan McKenna,0.132577,0.7817,-358,0.649123,-185,0.5
49,2023-05-29,Cleveland Guardians,Baltimore Orioles,Cam Gallagher,0.079245,0.6247,-166,0.545455,-120,0.5
